In [1]:
import pandas as pd  #csv-xlsm
import os            #directories
import time          #spend time
from collections import Counter  #magia de sumar diccionarios!
import math

In [2]:
### LIMPIEZA
### DIRECTORIOS
path = os.getcwd()
directory_files = '/OneDrive-2022-01-29/'

In [3]:
### FUNCION DE RECOLECCION DE LOS XLSM #########################################
### CONTADOR DE ARCHIVOS###
def preprocesamiento(path,directory_files):
    '''
        Elije del excel las filas y columnas de fecha, hora, codigo y nombre.
        Este proceso lo hace para todos los excel en un directorio.
        Obs: Todos deben tener el mismo formato.

        path: directorio principal (pwd)
        directory_files: directorio específico donde se encuentran los excel.

    '''
    count = 0
    ### NUEVOS NOMBRES PARA LAS COLUMNAS DE LOS XLSM ###############################
    column_names = ["fecha","horas","cod","nombre"]
    archivos = list()
    ID = list()
    start_time = time.time()

    for subdir, dirs, files in os.walk(path+directory_files):
        for file in files:
            count += 1
            try:
                print(file)
                ID.append(file)
                df = pd.read_excel(path+directory_files+file, index_col = False, sheet_name = None)
                tabla = df['HH'].iloc[10:149,1:5]
                tabla.columns = column_names

                archivos.append(tabla)

        ## If the file is not valid, skip it
            except ValueError:
                continue

            print("Total Number of XLSM Files")
            print(count)
            print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))
    return archivos, ID


#### CAPTURAR HORAS Y FILAS PARA SUMAR POSTERIORMENTE ##########################
def periodos_tiempo(archivo_i):
    '''
        Almacena las filas donde se encuentran anotadas las horas de trabajo.
    '''
    i = 0
    rangos_de_tiempo = list()
    ##### tener los tiempos ######
    for dia in archivo_i.fecha:

        if not pd.isna(dia):
            rangos_de_tiempo.append(i)
        i += 1
    ###############################

    ultimo_valor = len(archivo_i.fecha) - rangos_de_tiempo[-1]
    rangos_de_tiempo.append(ultimo_valor)
    return rangos_de_tiempo

def suma_horas_por_dia(rangos_de_tiempo,archivo_i,codigos):
    '''
        Suma las horas trabajadas por dia y almacena la información en un diccionario
        donde la llave es el código de obra y los valores la suma de las horas trabajadas.
    '''
    # partida
    lim_inf = 0
    lim_sup = 2

    #dict_cod = {}

    horas_dia = []

    for i in range(len(rangos_de_tiempo)-1):

        # agarro el intervalo
        intervalo = rangos_de_tiempo[lim_inf:lim_sup]
        # me adelanto para la modificacion del intervalo en la siguiente iteracion
        lim_inf = i + 1
        lim_sup = i + 3

        dict_cod = reset_codigos(codigos)

        for j in range(intervalo[0],intervalo[1]):

            if not math.isnan(archivo_i.cod.iloc[j]):
                dict_cod[str(archivo_i.cod.iloc[j])] =+ archivo_i.horas.iloc[j] + dict_cod[str(archivo_i.cod.iloc[j])]

        horas_dia.append({x:y for x,y in dict_cod.items() if y!=0})
    
    return horas_dia
################################################################################
def reset_codigos(codigos):
    '''
        Resetea los valores del diccionario de codigos a cero.
    '''
    # reset del diccionario
    dict_cod = {}

    for i in range(1,len(codigos.codigos)):
        dict_cod[str(codigos.codigos[i])] = 0

    dict_cod["9020003"] = 0
    dict_cod["1021065"] = 0
    dict_cod["1021019"] = 0
    dict_cod["2021035"] = 0
    dict_cod["9020001"] = 0
    dict_cod["5021012"] = 0
    dict_cod["1021063"] = 0
    dict_cod["9020004"] = 0
    dict_cod["6315"] = 0
    dict_cod["1021076"] = 0
    dict_cod["6316"] = 0
    dict_cod["9345"] = 0
    return dict_cod

################################################################################
def magia(horas):
    aux = Counter(horas[0])
    for i in range(1,len(horas)):
        aux = aux + Counter(horas[i])
    return aux

In [4]:
PATH="codigos.csv"
codigos = pd.read_csv(PATH, index_col = False)
archivos,ID = preprocesamiento(path,directory_files)
L = len(archivos)

HN-1221-TVK.xlsm


/Users/rodolfo.lobo/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Total Number of XLSM Files
1
--- Data loaded. Loading time: 0.7745490074157715 seconds ---
HN-1221-CHP.xlsm
Total Number of XLSM Files
2
--- Data loaded. Loading time: 1.080265998840332 seconds ---
HN-1221-DMA.xlsm
Total Number of XLSM Files
3
--- Data loaded. Loading time: 1.3322558403015137 seconds ---
HN-1221-FBC.XLSM
Total Number of XLSM Files
4
--- Data loaded. Loading time: 1.6063659191131592 seconds ---
HN-1221-CRH.xlsm
Total Number of XLSM Files
5
--- Data loaded. Loading time: 1.9266128540039062 seconds ---
HN-1221-SLF.xlsm
Total Number of XLSM Files
6
--- Data loaded. Loading time: 2.17344069480896 seconds ---
HN-1221-COL.XLSM
Total Number of XLSM Files
7
--- Data loaded. Loading time: 2.4651458263397217 seconds ---
HN-1221-CAV.xlsm
Total Number of XLSM Files
8
--- Data loaded. Loading time: 2.7194650173187256 seconds ---
HN-1221-JGM.xlsm
Total Number of XLSM Files
9
--- Data loaded. Loading time: 2.97357177734375 seconds ---
HN-1221-EAE.XLSM
Total Number of XLSM Files
10
---

In [5]:
horas_mes_de_los_trabajadores = list()
horas_dia_de_los_trabajadores = list()


for i in range(L):
    
    ### Probando para un solo archivo!
    print("Excel: ")
    print(ID[i])
    print()
    
    archivo_i = archivos[i] #esto tiene que estar dentro de un loop que recorra los archivos
    rangos_de_tiempo = periodos_tiempo(archivo_i)
    horas_dia = suma_horas_por_dia(rangos_de_tiempo,archivo_i,codigos) # es una lista de diccionarios
    horas_mes =  magia(horas_dia)
    horas_mes_de_los_trabajadores.append(horas_mes)            # es una lista de diccionarios
    horas_dia_de_los_trabajadores.append(horas_dia)
    
# todos los trabajadores juntos!
horas_mes_total = magia(horas_mes_de_los_trabajadores)

Excel: 
HN-1221-TVK.xlsm

Excel: 
HN-1221-CHP.xlsm

Excel: 
HN-1221-DMA.xlsm

Excel: 
HN-1221-FBC.XLSM

Excel: 
HN-1221-CRH.xlsm

Excel: 
HN-1221-SLF.xlsm

Excel: 
HN-1221-COL.XLSM

Excel: 
HN-1221-CAV.xlsm

Excel: 
HN-1221-JGM.xlsm

Excel: 
HN-1221-EAE.XLSM



In [6]:
len(horas_mes_de_los_trabajadores)

10

In [7]:
horas_mes_de_los_trabajadores

[Counter({'9020003': 1.5,
          '6264': 7.0,
          '5061': 2.5,
          '6305': 17.0,
          '6298': 7.5,
          '6266': 4.5,
          '6236': 4.5,
          '6296': 3,
          '6299': 3,
          '6307': 24.0,
          '6289': 10.5,
          '1021063': 2,
          '6252': 5.5,
          '6274': 1,
          '6309': 6,
          '9020004': 0.5,
          '9020002': 4,
          '9040': 2.5,
          '6242': 2,
          '6291': 2,
          '9020001': 87.0,
          '6311': 0.5,
          '6315': 8}),
 Counter({'9020001': 43.5,
          '6281': 8.5,
          '6285': 1.5,
          '6290': 1.5,
          '6310': 18.5,
          '9537': 13.0,
          '9020003': 19.5,
          '6292': 3.5,
          '6297': 2.5,
          '6313': 27.5,
          '5058': 0.5,
          '5060': 2.0,
          '6287': 4.0,
          '1018024': 1.5,
          '1021007': 7.5,
          '1021042': 15.0,
          '1021072': 16.5,
          '6301': 0.5,
          '6307': 3.5,
      

In [8]:
horas_mes_total

Counter({'9020003': 70.5,
         '6264': 21.0,
         '5061': 2.5,
         '6305': 73.5,
         '6298': 53.5,
         '6266': 22.5,
         '6236': 4.5,
         '6296': 4,
         '6299': 86.0,
         '6307': 36.5,
         '6289': 119.0,
         '1021063': 2,
         '6252': 18.5,
         '6274': 1,
         '6309': 41.0,
         '9020004': 65.5,
         '9020002': 22.0,
         '9040': 2.5,
         '6242': 2,
         '6291': 7,
         '9020001': 165.0,
         '6311': 73.5,
         '6315': 8,
         '6281': 8.5,
         '6285': 22.0,
         '6290': 1.5,
         '6310': 27.5,
         '9537': 19.5,
         '6292': 12.5,
         '6297': 3.5,
         '6313': 103.0,
         '5058': 0.5,
         '5060': 32.0,
         '6287': 23.0,
         '1018024': 1.5,
         '1021007': 7.5,
         '1021042': 20.5,
         '1021072': 16.5,
         '6301': 2.5,
         '8021012': 2,
         '1021048': 2,
         '1021076': 3,
         '3090': 13.5,
         

In [ ]:
## Ordenados de menor a mayor!

In [10]:
{k: v for k, v in sorted(horas_mes_total.items(), key=lambda item: item[1])}

{'5058': 0.5,
 '6274': 1,
 '1021070': 1,
 '6290': 1.5,
 '1018024': 1.5,
 '1021063': 2,
 '6242': 2,
 '8021012': 2,
 '1021048': 2,
 '5061': 2.5,
 '9040': 2.5,
 '6301': 2.5,
 '1021076': 3,
 '1021071': 3,
 '6297': 3.5,
 '6296': 4,
 '6236': 4.5,
 '9345': 6,
 '1020059': 6.5,
 '6291': 7,
 '1021065': 7.0,
 '2021032': 7,
 '1021007': 7.5,
 '6306': 7.5,
 '6315': 8,
 '6281': 8.5,
 '4012': 8.5,
 '6198': 9,
 '6280': 9,
 '6295': 9,
 '1021040': 9,
 '1021068': 10,
 '1021019': 10.0,
 '2021034': 10,
 '6238': 10,
 '2021009': 11,
 '5021011': 11,
 '6292': 12.5,
 '5045': 13.0,
 '3090': 13.5,
 '6316': 15,
 '2021033': 16.0,
 '1021072': 16.5,
 '4017': 17.5,
 '6252': 18.5,
 '9537': 19.5,
 '1021042': 20.5,
 '6264': 21.0,
 '5047': 21.5,
 '9020002': 22.0,
 '6285': 22.0,
 '6266': 22.5,
 '6287': 23.0,
 '2021035': 24.5,
 '6279': 25.5,
 '6314': 26.5,
 '6310': 27.5,
 '9543': 28.0,
 '5060': 32.0,
 '6307': 36.5,
 '6309': 41.0,
 '6259': 46.5,
 '1021074': 49.0,
 '6298': 53.5,
 '5021012': 61.0,
 '6312': 63.0,
 '9020004': 65.